In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import math
import glob
import json
import skimage
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
from google.colab.patches import cv2_imshow

In [ ]:
path = '/content/drive/MyDrive/CV/project'
dataset_path = '/content/Dataset'
patterns_path = '/content/Patterns'

In [ ]:
!unzip /content/drive/MyDrive/CV/project/Dataset.zip -d Dataset

In [ ]:
!unzip /content/drive/MyDrive/CV/project/Patterns.zip -d Patterns

Archive:  /content/drive/MyDrive/CV/project/Patterns.zip
  inflating: Patterns/AYLIN.tif      
  inflating: Patterns/Orlando_60x60-1.tif  
  inflating: Patterns/Orlando_60x60-10.tif  
  inflating: Patterns/Orlando_60x60-2.tif  
  inflating: Patterns/Orlando_60x60-3.tif  
  inflating: Patterns/Orlando_60x60-5.tif  
  inflating: Patterns/Orlando_60x60-6.tif  
  inflating: Patterns/Orlando_60x60-7.tif  
  inflating: Patterns/Orlando_60x60-8.tif  
  inflating: Patterns/Orlando_60x60-9.tif  
  inflating: Patterns/REJINA.tif     
  inflating: Patterns/sama.tif       


In [ ]:
dataset_images = glob.glob(dataset_path + '/*.jpg') + glob.glob(dataset_path + '/*.png') + glob.glob(dataset_path + '/*.bmp')
dataset_jsons = glob.glob(dataset_path + '/*.json')
patterns_images = glob.glob(patterns_path + '/*.tif') 

In [ ]:
images = []
for im_path in dataset_images:
  im = cv2.imread(im_path)
  images.append(im)

dataset_info = []
for d_path in dataset_jsons:
  with open(d_path) as f:
    info = json.load(f, encoding='utf8')
    dataset_info.append(info)

patterns = []
for p in patterns_images:
  pat = cv2.imread(p)
  patterns.append(pat)

In [ ]:
lk_params = dict(winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners = 20,
                    qualityLevel = 0.3,
                    minDistance = 10,
                    blockSize = 7 )


trajectory_len = 20
detect_interval = 1
trajectories = []
frame_idx = 0


# cap = cv2.VideoCapture(0)




    # start time to calculate FPS
    # start = time.time()

# suc, frame = cap.read()
frame = patterns[0]
frame_gray = cv2.cvtColor(images[0], cv2.COLOR_BGR2GRAY)
img = frame.copy()
prev_gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
if len(trajectories) > 0:
    img0, img1 = prev_gray, frame_gray
    p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)
    p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
    p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
    d = abs(p0-p0r).reshape(-1, 2).max(-1)
    good = d < 1
    new_trajectories = []
    # Get all the trajectories
    for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
        if not good_flag:
            continue
        trajectory.append((x, y))
        if len(trajectory) > trajectory_len:
            del trajectory[0]
        new_trajectories.append(trajectory)
        # Newest detected point
        cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)
    trajectories = new_trajectories
    # Draw all the trajectories
    cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
    cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
# Update interval - When to update and detect new features
if frame_idx % detect_interval == 0:
    mask = np.zeros_like(frame_gray)
    mask[:] = 255
    # Lastest point in latest trajectory
    for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
        cv2.circle(mask, (x, y), 5, 0, -1)
    # Detect the good features to track
    p = cv2.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
    if p is not None:
        # If good features can be tracked - add that to the trajectories
        for x, y in np.float32(p).reshape(-1, 2):
            trajectories.append([(x, y)])
frame_idx += 1
prev_gray = frame_gray
# # End time
# end = time.time()
# # calculate the FPS for current frame detection
# fps = 1 / (end-start)

# Show Results
# cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2_imshow( img)
cv2_imshow(mask)
    # if cv2.waitKey(10) & 0xFF == ord('q'):
    #     break


# cap.release()
# cv2.destroyAllWindows()